# Testing/Troubleshooting cell

In [ ]:
from qiskit import QuantumCircuit#, Aer, execute #, transpile #, assemble, execute
from qiskit_aer import AerSimulator  # Use AerSimulator for better performance
# from qiskit.providers.aer import Aer
from qiskit.visualization import plot_histogram

qunatum_circuit = QuantumCircuit(3, 3)
# qunatum_circuit.h(0)  # Apply Hadamard gate to qubit0
# qunatum_circuit.cx(0, 1)  # Apply CNOT gate with qubit0 as control and qubit1 as target
qunatum_circuit.x(0)
qunatum_circuit.x(1)
qunatum_circuit.measure([0, 1, 2], [0, 1, 2])  # Measure both qubits
#qunatum_circuit.draw('mpl')  # Draw the circuit
#print(qunatum_circuit.num_qubits)

# Run single shot measurement
simulator = AerSimulator()
job = simulator.run(qunatum_circuit, shots=1, memory=True)

# Get the result
result = job.result()
memory = result.get_memory()  # Gets the single shot result
counts = result.get_counts()

# Print the measurement results
print("Measurement counts:")
print(counts)
print(memory[0][0])

# Optional: Print as percentages
print("\nMeasurement probabilities:")
total_shots = sum(counts.values())
for bitstring, count in counts.items():
    probability = (count / total_shots) * 100
    print(f"{bitstring}: {probability:.2f}%")


: 

# Quantum effects for Ludo Tokens

In [ ]:
def bitstring_to_int(bitstring, sign):
    """Convert a bitstring to an integer."""
    return int(bitstring, 2)*sign

def token_circuit_initialization():
    qc = QuantumCircuit(6, 6)  # Create a quantum circuit with 6 qubits and 6 classical bits
    #Apply series of gate to attenuate probability of '000' and '111'

    dice_qc = QuantumCircuit(3, 3)  # Create a quantum circuit for dice with 3 qubits and 3 classical bits
    # Apply Hadamard gate to all qubits to create superposition
    dice_qc.h([0, 1, 2])
    return qc

def blockade_entanglement(qc1, qc2):
    """"Create a new quantum circuit with qc1.num_qubits + qc2.num_qubits qubits and qc1.num_clbits + qc2.num_clbits classical bits from 2 circuits.
    This is to simulate the entanglement between two tokens triggered when two tokens from a player form create a blockade. The two tokens can be from the same player or not.
        qc1 (QuantumCircuit): The first quantum circuit.
        qc2 (QuantumCircuit): The second quantum circuit.
    Returns:
        QuantumCircuit: A new quantum circuit that combines qc1 and qc2 with a CNOT gate to introduce entanglement between circuits.
    """""
    total_num_qubits = qc1.num_qubits + qc2.num_qubits
    total_num_clbits = qc1.num_clbits + qc2.num_clbits
    combined_qc = QuantumCircuit(total_num_qubits, total_num_clbits)
    # Add first circuit to qubits 0 and qc1.num_qubits
    combined_qc.compose(qc1, qubits=[0, qc1.num_qubits], inplace=True)
    # combined_qc.compose(qc1, clbits=[0, qc1.num_clbits], inplace=True)
    combined_qc.barrier()  # Add a barrier for clarity
    # Add second circuit to qubits qc1.num_qubits and qc1.num_qubits + qc2.num_qubits
    combined_qc.compose(qc2, qubits=[qc1.num_qubits, qc1.num_qubits + qc2.num_qubits], inplace=True)
    # combined_qc.compose(qc2, clbits=[qc1.num_clbits, qc1.num_clbits + qc2.num_clbits], inplace=True)
    combined_qc.barrier()  # Add a barrier for clarity
    # Add a CNOT gate between the first qubit of qc1 and the first qubit of qc2 to introduce entanglement between the two circuits
    combined_qc.cx(0, qc1.num_qubits)
    return combined_qc

def measure_blockade_entanglement(qc):
    """Measure the entanglement between two circuits by measuring the first qubit of each circuit.
    Args:
        qc (QuantumCircuit): The quantum circuit with entanglement.
    Returns:
        QuantumCircuit: A new quantum circuit with measurements of the first qubit of each circuit.
    """
    for i in range(qc.num_qubits):
        qc.measure(i, i)

    # Run single shot measurement
    simulator = AerSimulator()
    job = simulator.run(qunatum_circuit, shots=1, memory=True)

    # Get the result
    result = job.result()
    memory = result.get_memory()  # Gets the single shot result

    nb_of_tokens = qc.num_qubits // 6

    measurment_per_token = [] # Stores the state for each token
    for token in range(0, nb_of_tokens, 6):
        measurment_per_token.append(memory[0][token:token + 6])  # Since each token has 6 qubits
    
    dice_results = []  # Stores the dice results for each token as tuples of (dice 1, dice 2)
    # Need to convert each dice into a number
    for i in range(nb_of_tokens):
        dice_results = (bitstring_to_int(measurment_per_token[i][0:3], 1), bitstring_to_int(measurment_per_token[i][4:6]), 1) # Adds positive values of the dice
    return dice_results

def measure_blockade_anti_entanglement(qc):
    """Measure the entanglement between two circuits by measuring the first qubit of each circuit.
    Args:
        qc (QuantumCircuit): The quantum circuit with entanglement.
    Returns:
        QuantumCircuit: A new quantum circuit with measurements of the first qubit of each circuit.
    """
    for i in range(qc.num_qubits):
        qc.measure(i, i)

    # Run single shot measurement
    simulator = AerSimulator()
    job = simulator.run(qunatum_circuit, shots=1, memory=True)

    # Get the result
    result = job.result()
    memory = result.get_memory()  # Gets the single shot result

    nb_of_tokens = qc.num_qubits // 6

    measurment_per_token = [] # Stores the state for each token
    for token in range(0, nb_of_tokens, 6):
        measurment_per_token.append(memory[0][token:token + 6])  # Since each token has 6 qubits
    
    dice_results = []  # Stores the dice results for each token as tuples of (dice 1, dice 2)
    # Need to convert each dice into a number
    for i in range(nb_of_tokens):
        if i == nb_of_tokens - 1: # Last token is anti-entangled
            dice_results = (bitstring_to_int(measurment_per_token[i][0:3], -1), bitstring_to_int(measurment_per_token[i][4:6]), -1)
        else:
            dice_results = (bitstring_to_int(measurment_per_token[i][0:3], 1), bitstring_to_int(measurment_per_token[i][4:6]), 1) # Adds positive values of the dice
    return dice_results

def probability_boost(qc):
    # Apply series of gate to boost probability of '110'
    amplification_gates_seq = QuantumCircuit(3, 3)  # Create a quantum circuit of 3 qubits and 3 classical bits with sequence of gate used to amplifying state '110' through Grover's algorithm. 

    # Apply Hadamard gate to all qubits to create superposition
    amplification_gates_seq.h([0, 1, 2])
    # Apply X gate to the target state '110'
    amplification_gates_seq.x(2)  # Apply X gate to qubit 2
    amplification_gates_seq.x(1)  # Apply X gate to qubit 1
    amplification_gates_seq.ccx(0, 1, 2)  # Toffoli gate
    amplification_gates_seq.z(2)  # Apply phase flip to qubit 2
    amplification_gates_seq.ccx(0, 1, 2)  # Uncompute
    amplification_gates_seq.x(0)  # Unflip qubit 0
    amplification_gates_seq.x(1)  # Unflip qubit 1

    # Step 3: Diffusion operator (amplification)
    amplification_gates_seq.h([0, 1, 2])
    amplification_gates_seq.x([0, 1, 2])
    amplification_gates_seq.h(2)
    amplification_gates_seq.ccx(0, 1, 2)
    amplification_gates_seq.h(2)
    amplification_gates_seq.x([0, 1, 2])
    amplification_gates_seq.h([0, 1, 2])

    nb_tokens = qc.num_qubits//6
    for i in range (0, nb_tokens, 6):
        qc.compose(amplification_gates_seq, qubits=[i, i + qc.num_qubits], inplace=True)
    return qc

def probability_drop(qc):
    # Apply series of gate to attenuate probability of '110' or increase probability of '001'
    return qc


Note: you may need to restart the kernel to use updated packages.


In [17]:
print(int('01001', 2))

9
